In [1]:
import time
import os
import tweepy
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import time
from textblob import TextBlob
import re

In [2]:
current_path = os.path.abspath(os.getcwd())

In [4]:
def api_setting():
    APIKEY = '59bl4gUSYLkiHUgLAYS4Yt6Vv'
    APIKeySecret = 'TPuFWIM2mBdOaEzY1Gwe18C2bouFA7yESV8wMMlQOTIPPd1utG'
    AccessToken = '1450488095774846980-3gPYJk1QPJtriIIRIYXjLIjWqyKYUo'
    AccessTokenSecret = 'aI3DBsJcptmHyWTx1Xn5h3kdA6L297Gw7cQhaiurseBDa'
    auth = tweepy.OAuthHandler(APIKEY,APIKeySecret)
    auth.set_access_token(AccessToken,AccessTokenSecret)
    api = tweepy.API(auth)
    return api
def screen_names():
    names = ['metmuseum', 'MuseumModernArt', 'Guggenheim','whitneymuseum','brooklynmuseum',\
                'TheJewishMuseum','FotografiskaNY','MerchantsHouse', 'newmuseum',\
                'frickcollection','MorganLibrary','NeueGalerieNY','QueensMuseum','AMNH',\
                'BronxMuseum','MADmuseum','IntrepidMuseum','museumofsex','cooperhewitt','elmuseo',\
                'FolkArtMuseum','NYHistory','studiomuseum','RubinMuseum','AmerIndianNYC',\
                'AsiaSocietyNY','LeslieLohman','mocanyc','drawingcenter','NoguchiMuseum',\
                'Sept11Memorial','met_cloisters','museumatFIT','GrandCentralNYC','FederalHallNPS','rockcenternyc','OneWorldNYC','StJohnDivineNYC','BrooklynBotanic',\
               'MovingImageNYC','EmpireStateBldg','yankeestadium','911TribMuseum','RichmondTownNYC','StPatsNYC','AMNH','iAliceAusten',\
               'NYBG','BrooklynKids','LincolnCenter','ApolloTheater','BronxZoo','ArmstrongHouse','CentralParkNYC','carnegiehall','StatueEllisNPS','NYTransitMuseum',\
               'frauncestavern','AFBurialGrndNPS','StonewallNps','bbpnyc','TheShedNY','centralparkzoo','TheSeaportNYC','prospect_park','TRBirthplaceNPS','LunaParkNYC',\
               'highlinenyc','TheGarden','TimesSquareNYC','brooklynmuseum','queensbotanicl']
    return names
    
def name_correction(name):
    if name == 'StPatsNYC':
        name = 'St. Patrick’s Cathedral'
    elif name == 'apollotheater':
        name = 'Apollo Theater'
    elif name == 'Statue of Liberty NM':
        name = 'Statue of Liberty'
    elif name == 'NY Transit Museum':
        name = 'New York Transit Museum'    
    elif name == 'AfricanBurialGround':
        name = 'African Burial Ground National Monument'
    elif name == 'T.R. Birthplace':
        name = 'Theodore Roosevelt Birthplace'
    elif name == 'Luna Park in Coney Island':
        name = 'Luna Park'
    elif name == 'General Grant NM':
        name = 'General Grant National Memorial'
    elif name == 'MSG':
        name = 'Madison Square Garden'
    elif name == 'MoMA The Museum of Modern Art':
        name = 'The Museum of Modern Art'
    elif name == 'The American Folk Art Museum is open!':
        name = 'American Folk Art Museum'
    elif name == 'Intrepid Sea, Air & Space Museum':
        name = 'Intrepid Museum'
    elif name == 'AmerIndianMuseumNYC':
        name = 'National Museum of the American Indian' 
    elif name == 'The Frick Collection':
        name = 'Frick Collection' 
    elif name == 'The Bronx Museum of the Arts':
        name = 'Bronx Museum of the Arts'
    elif name == '9/11 Memorial & Museum':
        name = 'National September 11 Memorial & Museum'
    elif name == 'Asia Society NY':
        name = 'Asia Society Museum'
    elif name == 'Fraunces Tavern® Museum':
        name = 'Fraunces Tavern Museum'

    return name

def findzip(name):
    locator = Nominatim(user_agent=f'measureyw3606')
    new_name = name_correction(name)
    if new_name[-3] == 'NYC' or new_name[-2] == 'NY':
        location = locator.geocode(f'{new_name}')
    else:
        location = locator.geocode(f'{new_name}, New York')
    # print(new_name)
    zipcode=location.address.split(', ')[-2][:5]
    return zipcode , new_name

def gatherdata(screen_names,api):
    actual_name = []
    place_name = []
    place_tweet_id = []
    average_likes = []
    average_reweets =[]
    zipcode = []
    for j in range(len(screen_names)):
        name = ''
        likes = []
        tweets = []
        retweet = []
        # if (j+1)%15 == 0:
        #     time.sleep(900)
    
        for i in tweepy.Cursor(api.user_timeline, screen_name=screen_names[j],tweet_mode = 'extended',exclude_replies=True).items(200):
            tweets.append(i.full_text)
            likes.append(i.favorite_count)
            retweet.append(i.retweet_count)
            # authorname.append(i.author.name)
            name = i.author.name
        df = pd.DataFrame({'tweets':tweets,'likes':likes,'retweet':retweet})
        df = df[~df.tweets.str.contains('RT')].reset_index(drop=True)
        place_name.append(name)
        place_tweet_id.append(screen_names[j])
        average_likes.append(np.mean(df.likes))
        average_reweets.append(np.mean(df.retweet))

        # locator = Nominatim(user_agent=f'measureyw3606')
        # new_name = name_correction(name)
        # location = locator.geocode(f'{new_name}')
        # print(new_name)
        # zipcode.append(location.address.split(', ')[-2][:5])
        zips, name = findzip(name)
        actual_name.append(name)
        zipcode.append(zips)
        numbersoftweets = len(df.likes)

    data = pd.DataFrame({'acutal_name':actual_name,'place_name':place_name,'zipcode':zipcode,'place_tweet_id':place_tweet_id,'average_likes':average_likes,\
                         'average_reweets':average_reweets})
    # data.to_csv('officialacocuntdata.csv', index = False)
    return data

def sentimental(data,api,path):
    positive_rate = []
    negative_rate = []
    j = 0
    for name in data.acutal_name:
        j+=1
        # print(name)
        tweets = tweepy.Cursor(api.search_tweets, q = name, lang = 'en',  tweet_mode="extended").items(200)
        if (j)%15 == 0:
            time.sleep(15*60)
        full = []
        for i in tweets:
            full.append(i.full_text)
        if full:
    
            df = pd.DataFrame({'tweets':full})      
            df = df[~df.tweets.str.contains('RT')].reset_index(drop=True)
        pos_num = 0
        neg_num = 0
        

        for tweet in df.tweets:

            text = tweet
            text = re.sub(r"http\S+", "", text)
            text = re.sub(r"#", "", text)
            text = re.sub(r"@", "", text)
            polarity = TextBlob(text)
            if polarity.sentiment[0] > 0 :
                pos_num += 1
            elif polarity.sentiment[0] < 0:
                neg_num += 1
        positive_rate.append(pos_num/(len(df.tweets)+0.0000001))
        negative_rate.append(neg_num/(len(df.tweets)+0.0000001))
    # s = pd.Series(data = positive_rate)    
    # df = pd.DataFrame({'acutal_name':actual_name,'place_name':place_name,'zipcode':zipcode,\
    #                      'place_tweet_id':place_tweet_id,'average_likes':average_likes,'average_reweets':average_reweets,\
    #                     'positive_rate': positive_rate})
    data['positive_rate'] = positive_rate
    data['negative_rate'] = negative_rate
    data.to_csv(path + '/twitter/attractions.csv', index = False)

In [3]:
def popularity_search(path = current_path):
    cur = time.time()
    last_mod = os.path.getctime(path + '/twitter/attractions.csv')
    current_time = time.time()
    difference = current_time - last_mod
    if difference > 604800:
    # if difference > 1:
        api = api_setting()
        names = screen_names()
        data = gatherdata(names,api)
        sentimental(data,api,path)
    

In [4]:
popularity_search()

The Metropolitan Museum of Art
The Museum of Modern Art
Guggenheim Museum
Whitney Museum
Brooklyn Museum
The Jewish Museum
Fotografiska New York
Merchant's House
New Museum
Frick Collection
The Morgan
Neue Galerie New York
Queens Museum
American Museum of Natural History
Bronx Museum of the Arts
Museum of Arts and Design
Intrepid Museum
Museum of Sex
Cooper Hewitt
El Museo del Barrio
American Folk Art Museum
New-York Historical Society
The Studio Museum in Harlem
Rubin Museum
National Museum of the American Indian
Asia Society Museum
Leslie-Lohman Museum of Art
Museum of Chinese in America
The Drawing Center
The Noguchi Museum
National September 11 Memorial & Museum
The Met: Cloisters
Museum at FIT
Grand Central Terminal
Federal Hall National Memorial
Rockefeller Center
One World Observatory
Cathedral of St. John the Divine
Brooklyn Botanic Garden
Museum of the Moving Image
Empire State Building
Yankee Stadium
9/11 Tribute Museum
Historic Richmond Town
St. Patrick’s Cathedral
American 